In [1]:
import sys
sys.version 

'3.10.14 | packaged by Anaconda, Inc. | (main, May  6 2024, 19:44:50) [MSC v.1916 64 bit (AMD64)]'

In [2]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [4]:
from langchain_community.chat_models.openai import ChatOpenAI
import os

gpt3 = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=os.getenv("OPENAI_API_KEY"))

In [8]:
class CustomStrOutputParser(StrOutputParser):
    def parse(self, output: str):
        print(output)
        return super().parse(output)

In [7]:
from typing import Optional, Callable
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

def create_prompt_chain(
    system_template: str, 
    human_template: str,
    model: Optional[ChatOpenAI] = None) -> Callable:
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_template), 
        ("human", human_template)
    ])
    return prompt | model | CustomStrOutputParser()

In [9]:
import re

# Step 1: 사용자 입력으로부터 검색 쿼리 생성
def create_search_query(user_input):
    query_template = """
    You are an expert in algorithms and data structures. Based on the user's input, generate a simple query to search for the key concept on Wikipedia. The input is: "{user_input}". Create the query to capture the essence of the user's question in simple keywords. 
    
    Please format your response as follows:
    Wikipedia Query: [Your query]
    """
    query_chain = create_prompt_chain(query_template, "{user_input}", gpt3)
    query = query_chain.invoke({"user_input": user_input})
    query_match = re.search(r'Wikipedia Query:\s*(.*)', query).group(1).strip()
    return query_match

In [10]:
# 예시 사용법
user_input = "I don't really understand what the Selection Sort algorithm is. Please explain it to me in detail and in an easy way."
search_query = create_search_query(user_input)
print(search_query)

Wikipedia Query: Selection Sort Algorithm
Selection Sort Algorithm


In [11]:
from langchain.document_loaders import WikipediaLoader
# Step 2: WikipediaLoader로 문서 검색
def search_wikipedia(query):
    loader =  WikipediaLoader(query=query, load_max_docs=2)  # 최대 2개의 문서 로드
    documents = loader.load()
    return documents

In [12]:
documents = search_wikipedia(search_query)

In [13]:
documents

[Document(metadata={'title': 'Selection sort', 'summary': 'In computer science, selection sort is an in-place comparison sorting algorithm. It has an O(n2) time complexity, which makes it inefficient on large lists, and generally performs worse than the similar insertion sort. Selection sort is noted for its simplicity and has performance advantages over more complicated algorithms in certain situations, particularly where auxiliary memory is limited.\nThe algorithm divides the input list into two parts: a sorted sublist of items which is built up from left to right at the front (left) of the list and a sublist of the remaining unsorted items that occupy the rest of the list. Initially, the sorted sublist is empty and the unsorted sublist is the entire input list. The algorithm proceeds by finding the smallest (or largest, depending on sorting order) element in the unsorted sublist, exchanging (swapping) it with the leftmost unsorted element (putting it in sorted order), and moving the

In [14]:
def escape_latex(text):
    # LaTeX 수식을 찾아 이스케이프 처리 (줄바꿈 포함)
    def replace_latex(match):
        return '{{' + match.group(1) + '}}' + match.group(2)
    
    # {\\displaystyle ...}\n 패턴을 찾아 이스케이프 처리
    escaped = re.sub(r'(\{\\displaystyle [^}]+\})(\n?)', replace_latex, text)
    
    # 남은 중괄호들을 이스케이프 처리
    escaped = escaped.replace("{", "{{").replace("}", "}}")
    
    return escaped

In [20]:
# Step 3: 검색된 결과를 바탕으로 답변 생성
def generate_answer(documents, user_input):
    qa_template = """
    Based on the Wikipedia content provided below, answer the user's question in a detailed and easy-to-understand way.
    
    Wikipedia content: {document_content}
    
    The user asked: {user_input}
    
    Provide a thorough and easy-to-follow explanation, ensuring the response is detailed and simplified for clear understanding using the Wikipedia data. 
    Additionally, use examples to make the explanation clearer and more relatable.
    """
    qa_chain = create_prompt_chain(qa_template, "{user_input}\n{document_content}", gpt3)
    document_content = " ".join([escape_latex(doc.page_content) for doc in documents])
    
    # documents는 WikipediaLoader로부터 로드된 문서들이며, 이를 기반으로 답변 생성
    answer = qa_chain.invoke({"user_input": user_input, "document_content": document_content})
    return answer

In [23]:
result = generate_answer(documents, user_input)

Selection Sort is a simple and easy-to-understand sorting algorithm used in computer science. It is an in-place comparison-based sorting algorithm, meaning it doesn't require extra memory space for sorting. Here's a detailed explanation of how Selection Sort works:

1. **Dividing the Input List**: The algorithm starts by dividing the input list into two parts:
   - Sorted Sublist: Initially empty, it will gradually build up from left to right.
   - Unsorted Sublist: Initially, the entire input list is unsorted.

2. **Sorting Process**:
   - The algorithm repeatedly finds the smallest (or largest) element from the unsorted sublist.
   - It then swaps this element with the leftmost element of the unsorted sublist, putting it in its correct sorted position.
   - The boundaries of the sorted and unsorted sublists are adjusted accordingly.

3. **Example**:
   Let's sort an array `[64, 25, 12, 22, 11]` using Selection Sort:
   - Find the minimum element in the entire array and swap it with t

In [24]:
from IPython.display import Markdown
Markdown(result)

Selection Sort is a simple and easy-to-understand sorting algorithm used in computer science. It is an in-place comparison-based sorting algorithm, meaning it doesn't require extra memory space for sorting. Here's a detailed explanation of how Selection Sort works:

1. **Dividing the Input List**: The algorithm starts by dividing the input list into two parts:
   - Sorted Sublist: Initially empty, it will gradually build up from left to right.
   - Unsorted Sublist: Initially, the entire input list is unsorted.

2. **Sorting Process**:
   - The algorithm repeatedly finds the smallest (or largest) element from the unsorted sublist.
   - It then swaps this element with the leftmost element of the unsorted sublist, putting it in its correct sorted position.
   - The boundaries of the sorted and unsorted sublists are adjusted accordingly.

3. **Example**:
   Let's sort an array `[64, 25, 12, 22, 11]` using Selection Sort:
   - Find the minimum element in the entire array and swap it with the first element: `[11, 25, 12, 22, 64]`
   - Find the minimum element in the remaining array and swap it with the second element: `[11, 12, 25, 22, 64]`
   - Repeat this process until the whole array is sorted: `[11, 12, 22, 25, 64]`

4. **Efficiency**:
   - Selection Sort has a time complexity of O(n^2), which means its efficiency decreases for large lists compared to more advanced algorithms like merge sort or quicksort.
   - However, Selection Sort can be advantageous in situations where auxiliary memory is limited due to its in-place sorting nature.

In summary, Selection Sort is a straightforward algorithm that repeatedly selects the smallest element from the unsorted part and moves it to the sorted part, gradually sorting the entire list. While it may not be the most efficient for large datasets, its simplicity and minimal memory usage make it a viable option in certain scenarios.